In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *


ModuleNotFoundError: No module named 'pyspark'

In [4]:
!pyspark --version

24/02/18 09:32:07 WARN Utils: Your hostname, Pratiks-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.0.0.43 instead (on interface en0)
24/02/18 09:32:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 3.5.0
      /_/
                        
Using Scala version 2.12.18, OpenJDK 64-Bit Server VM, 11.0.15
Branch HEAD
Compiled by user ubuntu on 2023-09-09T01:53:20Z
Revision ce5ddad990373636e94071e7cef2f31021add07b
Url https://github.com/apache/spark
Type --help for more information.


In [8]:

# Create a SparkSession
spark = SparkSession.builder \
    .appName("PySpark Data Analysis") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/18 09:28:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [9]:
# Load your dataset (replace 'your_dataset.csv' with your actual dataset file)
data = spark.read.csv('sample_data_test.csv', header=True, inferSchema=True)


In [10]:
# Display the schema of the dataset
data.printSchema()

# Show the first few rows of the dataset
data.show(5)

root
 |-- Index: integer (nullable = true)
 |-- Customer Id: string (nullable = true)
 |-- First Name: string (nullable = true)
 |-- Last Name: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Phone 1: string (nullable = true)
 |-- Phone 2: string (nullable = true)
 |-- Email: string (nullable = true)
 |-- Subscription Date: date (nullable = true)
 |-- Website: string (nullable = true)

+-----+---------------+----------+---------+--------------------+----------------+--------------------+--------------------+--------------------+--------------------+-----------------+--------------------+
|Index|    Customer Id|First Name|Last Name|             Company|            City|             Country|             Phone 1|             Phone 2|               Email|Subscription Date|             Website|
+-----+---------------+----------+---------+--------------------+----------------+--------------------+

In [5]:
# Count the number of rows in the dataset
print("Total number of rows:", data.count())


Total number of rows: 2000000


In [6]:
cleaned_data = data.dropna()

# Count the number of rows after cleaning
print("Total number of rows after cleaning:", cleaned_data.count())


Total number of rows after cleaning: 2000000


In [12]:
# Wide transformation example
wide_df = data.groupBy("Country").count()
print("Wide Transformation Result:")
wide_df.show()

Wide Transformation Result:


+--------------------+-----+
|             Country|count|
+--------------------+-----+
|                Chad| 8134|
|            Anguilla| 8128|
|            Paraguay| 8130|
|               Macao| 8247|
|Heard Island and ...| 8082|
|               Yemen| 8058|
|             Senegal| 8181|
|             Tokelau| 8196|
|              Sweden| 8110|
|            Kiribati| 8210|
|French Southern T...| 8091|
|              Guyana| 8068|
|         Philippines| 8227|
|             Eritrea| 7974|
|              Jersey| 8011|
|            Djibouti| 8286|
|               Tonga| 8160|
|      Norfolk Island| 8113|
|            Malaysia| 8166|
|           Singapore| 8143|
+--------------------+-----+
only showing top 20 rows



In [11]:
# Narrow transformation example
narrow_df = data.filter(data["Country"] == "Costa Rica").select("First Name", "Last Name")
# Show results
print("Narrow Transformation Result:")
narrow_df.show()

Narrow Transformation Result:
+----------+---------+
|First Name|Last Name|
+----------+---------+
|      Gina|    Rocha|
|  Margaret|   Rogers|
|     Tanya| Franklin|
|     Steve|Valentine|
|     Tracy|   Briggs|
|  Benjamin|     Gray|
|    Colton| Mcintyre|
|      Gail|      Ray|
|    Brandi|Contreras|
|      Troy|  Sellers|
|     Robin|    Black|
|     Nancy|    Brown|
|    Ernest| Mcmillan|
|   Jasmine|   Norman|
|     Sonya|   Turner|
|     Jorge|    Mejia|
|     Chris|   Walter|
|     Penny|  Carlson|
|       Jay|   Booker|
|     David|  Lindsey|
+----------+---------+
only showing top 20 rows



In [13]:
# Get the number of partitions
num_partitions = data.rdd.getNumPartitions()
print("Number of partitions:", num_partitions)

Number of partitions: 8
